In [ ]:
!pip install requests
!pip install python-docx

In [1]:
import requests
import json
import base64
import os
import re
from docx import Document
from docx.shared import Inches



#Get_accesscode 用于向百度获取access token key，第一个参数user_api是创建的应用API，第二个参数user_key是创建应用的key

In [3]:
def get_accesscode(user_api:str,user_key:str)->str:
    url = "https://aip.baidubce.com/oauth/2.0/token?client_id="+user_api+"&client_secret="+user_key+"&grant_type=client_credentials"
    
    payload = json.dumps("")
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    
    response = requests.request("POST", url, headers=headers, data=payload)
    access_token = response.json()['access_token']
    return access_token

access_token_link = get_accesscode("xxx","xxx")

写入word文档


In [7]:
def write_into_new_word(heading:str,name:str,response):
    # 创建一个 Word 文档对象
    document = Document()
    # 添加标题
    document.add_heading(heading, 0)
    #添加段落
    document.add_paragraph("Subject: Chinese")
    document.add_paragraph("Level: Secondary 2 / S2 / F2")
    document.add_paragraph("Region: Hong Kong")
    document.add_paragraph("")

    
    #document.add_paragraph("Note: (a^2) mean the power of a is 2")
    #下面的for loop是讲扫描到的写入word文档，用不同的OCR写法不一样，可以先print一下
    #response.json去看看格式然后去改下面的格式
    for i in range(len(response.json()['words_result'])):
        document.add_paragraph(response.json()['words_result'][i]['words'])
    '''
    for i in range(len(response.json()['results'])):
        document.add_paragraph(response.json()['results'][i]['words']['word']) 
        #text = text + response.json()['results'][i]['words']['word'] +' '
    '''
    document.save(name)

往现有的word文档里面填写资料

In [8]:

def write_into_exist_word(name:str,response)->None:
    # 打开现有的 Word 文档
    document = Document(name)

    #document.add_paragraph("Note: (a^2) mean the power of a is 2")
    #下面的for loop是讲扫描到的写入word文档，用不同的OCR写法不一样，可以先print一下
    #response.json去看看格式然后去改下面的格式
    for i in range(len(response.json()['words_result'])):
        document.add_paragraph(response.json()['words_result'][i]['words'])
        
    '''
    for i in range(len(response.json()['results'])):
        document.add_paragraph(response.json()['results'][i]['words']['word'])
        #text = text + response.json()['results'][i]['words']['word'] +' '
    '''
    # 保存文档
    document.save(name)

查看文件路径是否按顺序

In [10]:
import os
import re

folder_path = 'Engwriting'

# 提取文件名中的数字部分并进行排序
def sort_key(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else float('inf')

# 获取文件列表并按数字排序
filenames = sorted(os.listdir(folder_path), key=sort_key)

# 打印排序后的文件名
for filename in filenames:
    print(filename)

img5.jpg
img11.jpg
img15.jpg
img19.jpg
img23.jpg
img27.jpg


In [ ]:
import os
#改成你个人的图片集
folder_path = 'Engwriting'
# 二进制方式打开图片文件
request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/accurate_basic"
#request_url每个服务都不一样，可以在百度的技术文档中找到相对应的url
First_control = True

for filename in sorted(os.listdir(folder_path), key=sort_key):
    f = open(folder_path+"/"+filename, 'rb')
    img = base64.b64encode(f.read())
    
    params = {"image":img,"language_type":"CHN_ENG","result_type":"big"}
    #前面向百度请求的token
    access_token = access_token_link

    request_url = request_url + "?access_token=" + access_token
    headers = {'content-type': 'application/x-www-form-urlencoded'}

    response = requests.post(request_url, data=params, headers=headers)
    if response:
        print (response.json())
    #第一次获取就写入并创建Word 文档
    if First_control:
        write_into_new_word('Engwriting','writing.docx',response)     
        First_control = False
    #第二次开始就加入现有文档，确保现有文档
    else:
        write_into_exist_word('writing.docx',response)